In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random, sys

text = """
'신의 퀴즈' 류덕환이 서유정을 의심했다.
29일 방송된 케이블채널 OCN 수목드라마 '신의 퀴즈:리부트' 6회에서는 요양병원을 찾아간 한진우(류덕환 분), 강경희(윤주희)의 모습이 그려졌다.
이날 사건의 모든 방향이 원장 신연화(서유정)가 있는 요양병원을 가리키고 있단 게 밝혀졌다. 강경희는 윤현종 누나에게 "현종 씨랑 지금 보신 분들 공통점 아냐. 같은 자살 사이트 회원이었고, 이 요양병원을 거쳐 간 후 실종됐다. 이 병원의 누군가와 관련되어 있을 확률이 높다"라고 밝혔다.
한진우는 신연화를 찾아가 "윤현종의 MAO-A에 대한 또 다른 견해가 있었냐. 특이점이나 합병증 여부"라며 "윤현종 씨에게 수술 자국이 있더라"라고 물었다. 이에 신연화는 "저희 병원엔 수술방이 없다. 당연히 뇌 수술도 하지 못한다"라고 말했다.
그러자 한진우는 "어떻게 알았냐. 뇌쪽 수술이란 거. 전 말씀 드린 적 없는데. 게다가 MAO-A는 뇌 수술과 직접적인 연관도 없고"라고 의심했다.
"""

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
print('사용되고 있는 문자의 수:',  chars )

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []

사용되고 있는 문자의 수: 163
사용되고 있는 문자의 수: ['\n', ' ', '"', "'", '(', ')', ',', '-', '.', '2', '6', '9', ':', 'A', 'C', 'M', 'N', 'O', '가', '간', '강', '같', '거', '건', '게', '견', '경', '고', '공', '과', '관', '국', '군', '그', '금', '나', '날', '냐', '널', '높', '뇌', '누', '는', '다', '단', '당', '대', '더', '덕', '데', '도', '됐', '되', '된', '드', '든', '들', '떻', '또', '라', '란', '랑', '러', '려', '련', '류', '률', '른', '를', '리', '린', '마', '말', '며', '모', '목', '못', '물', '밝', '방', '병', '보', '부', '분', '블', '사', '살', '서', '송', '수', '술', '습', '신', '실', '심', '씀', '씨', '아', '알', '았', '양', '어', '없', '었', '에', '엔', '여', '연', '와', '요', '우', '원', '유', '윤', '은', '을', '의', '이', '인', '일', '있', '자', '장', '저', '적', '전', '점', '접', '정', '졌', '종', '주', '즈', '증', '지', '직', '진', '쪽', '찾', '채', '쳐', '케', '퀴', '키', '통', '트', '특', '하', '한', '합', '해', '했', '향', '혀', '현', '혔', '화', '확', '환', '회', '후', '희', '히']


In [3]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

학습할 구문의 수: 158
텍스트를 ID 벡터로 변환합니다...


In [4]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

/var/folders/_z/yb3zdl1n7db9r20dj_c89fq00000gn/T/ipykernel_13413/3922026183.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/var/folders/_z/yb3zdl1n7db9r20dj_c89fq00000gn/T/ipykernel_13413/3922026183.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences)

In [5]:
# 모델 구축하기(LSTM)
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)

        #텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()



--------------------------------------------------
반복 = 1
2/2 [==============================] - 1s 12ms/step - loss: 5.0857

--- 다양성 =  0.2
--- 시드 = "요양병원을 가리키고 있단 게 밝혀졌다"
요양병원을 가리키고 있단 게 밝혀졌다목원러 마주든목서에A특국는윤직알진신주양트날6 C씀든견았,또신윤인또살드(직정리점
인리한공주쪽:희윤며졌는엔
환나날란심해어와려더혀강특른.른습점말통향못견못적널도관없확경었라 며모덕금률국씨서가더다드드N(단었뇌즈(률대퀴6특과 분해없강요자엔M인방저'덕관공류날병있)일란쪽고C점냐린"요A모혔장해O자데국원적확그정아며트유률엔단도,강여가우술M날서합마드진엔접를습환강9쪽:되냐한원심인덕일강9퀴관누리케알서실요병혔,덕N전.견회.통'C하른히M목점종됐(건O종졌여날데살았모방화금우그견보을살건게았접게뇌간,분있현었밝정경실(특진유누,트환의있강케여률을블강분보떻아희병되졌퀴씀랑러실인강에과블부널게데N견심습마졌현은당강향도 혔사O송증혔히A퀴6떻란심었견우A보진씀견우해:데들서)혔대고그 O:O윤았원도심N간에습심뇌해C)혔이9마특가부보실른군진가저마후물C률
린합-병혀마합우라

--- 다양성 =  0.5
--- 시드 = "요양병원을 가리키고 있단 게 밝혀졌다"
요양병원을 가리키고 있단 게 밝혀졌다확널엔금회강려유그송"화수경말며되향어은덕냐데대방른인환들마키견술사날병해아목못누유건알들찾진보증)키며높희러쳐밝들련환합과마없인의리혔6장일O떻서접엔적신환우연.장드한란한엔여누살됐트병-혔들N른요은련보금사며여마건련고A도알윤찾목접와A연못려인:주C공했고러된"후-가주혔나심주합보합원거뇌높됐경뇌와린견경떻윤과강단2며적널부N화마우송후현단서라방있송장분N률환나들확없엔블았쪽게우찾없실졌트데확양희있한자날밝분보증아과못윤어수러를쪽A인없었뇌데일간연심떻점특진쪽데적드접마모금씨M윤데거드 률희화대찾인관N서즈란일널률방련6견쳐다더뇌모키,유없게덕들혀M2살날는채윤사회유었접가케은적통합된지를일된았누하같게간당도적모N(통직M장환금M관화지블일관란관후요여술분며떻러

는 "어떻게 알았냐. 뇌쪽 수술이란                                                                                                                                                                                                                                                                                                                                                                                                                 

--- 다양성 =  0.5
--- 시드 = "는 "어떻게 알았냐. 뇌쪽 수술이란 "
는 "어떻게 알았냐. 뇌쪽 수술이란               다          이 현병                              률                        수 도                      모                       현             가              덕         현                                이       한                                                              이  씀                  수       강수             수                             .    수                                       .   

--- 다양성 =  1.0
--- 시드 = "는 "어떻게 알았냐. 뇌쪽 수술이란 "
는 "어떻게 알았냐. 뇌쪽 수술이란 건(리뇌는현 라공대 란라부과O일방적 높 또분 았날강을양른란C유없 술는정(다없